In [138]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

In [139]:
data= pd.read_csv('cloud.csv',nrows=500,index_col=0)

In [140]:
pd.isnull(data)

,vm_id,timestamp,cpu_usage,memory_usage,network_traffic,power_consumption,num_executed_instructions,execution_time,energy_efficiency,task_type,task_priority,task_status
0,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,True,False,False,False,False,False,False,False
2,False,False,True,False,False,False,False,False,False,False,False,False
3,False,False,False,False,True,False,False,False,False,False,False,False
4,False,False,False,True,True,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
495,False,False,False,False,False,False,True,False,True,False,False,False
496,False,False,False,False,False,True,False,False,False,False,False,False
497,False,True,False,True,False,False,False,False,False,True,False,False
498,False,False,False,False,False,False,False,False,False,False,False,False


In [141]:

data.drop(['vm_id','timestamp'],axis=1,inplace=True)


In [142]:
data.dropna(inplace=True)

In [143]:
data

,cpu_usage,memory_usage,network_traffic,power_consumption,num_executed_instructions,execution_time,energy_efficiency,task_type,task_priority,task_status
0,54.881350,78.950861,164.775973,287.808986,7527.0,69.345575,0.553589,network,medium,waiting
6,43.758721,22.456429,429.139640,272.960417,9008.0,60.153904,0.461246,compute,high,completed
9,38.344152,16.441881,779.791345,382.756696,2989.0,42.160847,0.139187,io,medium,running
10,79.172504,2.972252,926.374030,173.559325,8644.0,55.702673,0.779499,compute,medium,completed
12,56.804456,2.355626,722.552146,143.340399,9788.0,79.700756,0.944387,io,medium,waiting
...,...,...,...,...,...,...,...,...,...,...
485,60.411780,29.114555,671.696182,368.399816,9316.0,9.615111,0.571332,io,medium,completed
487,89.538588,58.398592,408.149530,210.676450,582.0,10.026512,0.259868,io,medium,waiting
491,59.223042,81.958323,606.801638,50.916785,5258.0,13.689116,0.194744,network,high,waiting
494,55.207828,56.407995,212.002158,29.931299,139.0,7.101163,0.199339,io,low,running


In [144]:
data['task_type'].unique()
data['task_type'].replace({'network':1,'compute':2,'io':3},inplace=True)
data['task_priority'].unique()
data['task_priority'].replace({'low':1,'medium':2,'high':3},inplace=True)
data['task_status'].unique()
data['task_status'].replace({'waiting':1,'running':2,'completed':3},inplace=True)

C:\Users\adity\AppData\Local\Temp\ipykernel_32384\956017799.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['task_type'].replace({'network':1,'compute':2,'io':3},inplace=True)
C:\Users\adity\AppData\Local\Temp\ipykernel_32384\956017799.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dat

In [145]:
data.drop(['task_type','task_priority','task_status'],axis=1,inplace=True)

In [146]:
data['cpu_usage'].std()

28.489758101240163

In [147]:
data['memory_usage'].std()

29.196124181190324

In [148]:
data['memory_usage'].mean()

49.8105496495476

In [149]:
data['cpu_usage'].mean()

46.10329593291688

In [150]:
scaler=StandardScaler()

In [151]:
scaler.fit(data)

StandardScaler()

In [152]:
scaled_feature=scaler.transform(data)

In [153]:
data=pd.DataFrame(scaled_feature,columns=data.columns[:])

In [154]:
data

,cpu_usage,memory_usage,network_traffic,power_consumption,num_executed_instructions,execution_time,energy_efficiency
0,0.308977,1.000888,-1.305191,0.352830,0.864538,0.594749,0.073711
1,-0.082526,-0.939537,-0.300134,0.247578,1.359198,0.278024,-0.234363
2,-0.273112,-1.146120,1.032973,1.025851,-0.651174,-0.341979,-1.308819
3,1.163996,-1.608764,1.590251,-0.457010,1.237621,0.124644,0.827392
4,0.376668,-1.629944,0.815361,-0.671212,1.619722,0.951567,1.377495
...,...,...,...,...,...,...,...
174,0.503641,-0.710849,0.622017,0.924085,1.462072,-1.463436,0.132904
175,1.528870,0.294975,-0.379934,-0.193911,-1.455123,-1.449261,-0.906202
176,0.461799,1.104186,0.375301,-1.326341,0.106682,-1.323055,-1.123468
177,0.320469,0.226604,-1.125647,-1.475094,-1.603087,-1.550062,-1.108141


In [155]:
X=data.drop(['cpu_usage','memory_usage'],axis=1)

In [156]:
X

,network_traffic,power_consumption,num_executed_instructions,execution_time,energy_efficiency
0,-1.305191,0.352830,0.864538,0.594749,0.073711
1,-0.300134,0.247578,1.359198,0.278024,-0.234363
2,1.032973,1.025851,-0.651174,-0.341979,-1.308819
3,1.590251,-0.457010,1.237621,0.124644,0.827392
4,0.815361,-0.671212,1.619722,0.951567,1.377495
...,...,...,...,...,...
174,0.622017,0.924085,1.462072,-1.463436,0.132904
175,-0.379934,-0.193911,-1.455123,-1.449261,-0.906202
176,0.375301,-1.326341,0.106682,-1.323055,-1.123468
177,-1.125647,-1.475094,-1.603087,-1.550062,-1.108141


In [157]:
Y=data['cpu_usage']

In [158]:
Y_memory=data['memory_usage']

In [159]:
data.describe()

,cpu_usage,memory_usage,network_traffic,power_consumption,num_executed_instructions,execution_time,energy_efficiency
count,1.790000e+02,1.790000e+02,1.790000e+02,1.790000e+02,1.790000e+02,1.790000e+02,1.790000e+02
mean,-4.961891e-18,2.629802e-16,-4.341654e-17,1.587805e-16,-1.376925e-16,-2.480945e-16,1.984756e-16
std,1.002805e+00,1.002805e+00,1.002805e+00,1.002805e+00,1.002805e+00,1.002805e+00,1.002805e+00
min,-1.580414e+00,-1.688011e+00,-1.927359e+00,-1.679421e+00,-1.641832e+00,-1.749760e+00,-1.760194e+00
25%,-8.617742e-01,-9.524202e-01,-7.058564e-01,-7.888608e-01,-7.680759e-01,-8.062399e-01,-9.118686e-01
50%,-1.302380e-01,3.940580e-02,-2.012566e-02,-1.387231e-01,-1.589787e-02,-4.407222e-02,-2.541177e-02
75%,7.365145e-01,8.910145e-01,7.118375e-01,8.275136e-01,8.665419e-01,9.153233e-01,8.700492e-01
max,1.863093e+00,1.702486e+00,1.869044e+00,1.852666e+00,1.659468e+00,1.646773e+00,1.537311e+00


In [160]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=101)

In [161]:
knn= KNeighborsRegressor(n_neighbors=7)

In [162]:
X

,network_traffic,power_consumption,num_executed_instructions,execution_time,energy_efficiency
0,-1.305191,0.352830,0.864538,0.594749,0.073711
1,-0.300134,0.247578,1.359198,0.278024,-0.234363
2,1.032973,1.025851,-0.651174,-0.341979,-1.308819
3,1.590251,-0.457010,1.237621,0.124644,0.827392
4,0.815361,-0.671212,1.619722,0.951567,1.377495
...,...,...,...,...,...
174,0.622017,0.924085,1.462072,-1.463436,0.132904
175,-0.379934,-0.193911,-1.455123,-1.449261,-0.906202
176,0.375301,-1.326341,0.106682,-1.323055,-1.123468
177,-1.125647,-1.475094,-1.603087,-1.550062,-1.108141


In [163]:
ohe=OneHotEncoder()
ohe.fit(X[['network_traffic','power_consumption','num_executed_instructions','execution_time','energy_efficiency']])

OneHotEncoder()

In [164]:
coulmns_trans=make_column_transformer((OneHotEncoder(categories=ohe.categories_),['network_traffic','power_consumption','num_executed_instructions','execution_time','energy_efficiency']),remainder='passthrough')

In [165]:
pipe=make_pipeline(coulmns_trans,knn)

In [166]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array([-1.92735866, -1.89479548, -1.87408103, -1.85393543, -1.83591527,
       -1.82577551, -1.815002  , -1.8146472 , -1.75532427, -1.69932395,
       -1.61154476, -1.59363959, -1.5537354 , -1.55109352, -1.5230178 ,
       -1.4829318 , -1.4511121 , -1.3625216...
        1.3534079 ,  1.36349371,  1.36851828,  1.37749489,  1.39477558,
        1.39641166,  1.41748581,  1.42557048,  1.42771621,  1.46247052,
        1.46924233,  1.47402086,  1.48100371,  1.49100499,  1.49415737,
        1.49725605,  1.50060958,  1.52229286,  1.53731121])]),
                                                  ['network_traffic',
                                                   'power_consumption',
                                                   'num_executed_instructions',
                                                   'execution_time',
                                                   'energy_efficiency'])])),
                ('kneighborsregressor', KNeighborsRegressor(n_neighbors=7))])

In [167]:
y_prediction=pipe.predict(X_test)

In [168]:
r2_score(y_test,y_prediction)


0.006914974526196382

In [169]:
knn=knn.fit(X_train,y_train)

In [170]:
pred=knn.predict(X_test)

In [171]:
pred

array([ 0.7557818 , -1.025671  ,  0.62933151,  0.10482451, -0.0959072 ,
       -0.28979314, -0.18022791, -0.75362693, -0.32545715, -0.72212671,
        0.07967929, -0.29837943, -0.35443442, -0.1645291 ,  0.07411817,
        0.19591453,  0.16941445,  0.05559431,  0.0984319 , -0.02613743,
        0.30190546, -0.2733623 , -0.60418776, -0.39521236, -0.17157358,
       -0.02570095, -0.48323942, -0.13853387,  0.71611584, -0.03229781,
        0.18632   , -0.24737029, -0.05113069,  0.49153923, -0.23881482,
        0.62262331,  0.30238518, -0.02429968,  0.15725453, -0.34357718,
       -0.24466257,  0.21274493, -0.16309795, -0.46759808,  0.06012643,
       -0.23862723, -0.84096816, -0.40538567,  0.54606778, -0.3952269 ,
       -0.51322636,  0.52959418,  0.53338152, -0.9202852 , -0.17154253,
        0.31221018,  0.02166677, -1.0188283 , -0.09077074,  0.09687126,
       -0.29005977, -0.16918717, -0.40231405,  0.00256102, -0.41662063,
        0.62382329, -0.53193647,  0.39363592, -0.3971962 , -0.27

In [172]:
print(mean_squared_error(y_test,pred))

1.4059067014628033


In [173]:
import pickle

In [174]:
pickle.dump(pipe,open('KNN.pkl','wb'))

In [175]:
pickle.dump(knn,open('KNN_model','wb'))

In [176]:
model_loaded=pickle.load(open('KNN_model','rb'))
a=model_loaded.predict(pd.DataFrame([[1.590251,-0.457010,1.237621,0.124644,0.827392]],columns=['network_traffic','power_consumption','num_executed_instructions','execution_time','energy_efficiency']))
print(a)

[0.02640722]


In [177]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, Y_memory, test_size=0.3, random_state=101)

In [178]:
knn1= KNeighborsRegressor(n_neighbors=7)

In [179]:
knn1=knn1.fit(X_train1,y_train1)

In [180]:
pred1=knn1.predict(X_test1)

In [181]:
print(mean_squared_error(y_test1,pred1))

1.0785328918511778


In [182]:
pickle.dump(knn1,open('KNN_model2','wb'))